In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.cluster import KMeans

In [2]:
Подвести итоги розыгрыша (
    конверсия, 
    кто был участниками, 
    какой инкрементальный эффект по основным показателям
).
 
Условия розыгрыша: 
    Соверши покупку на сумму от 5000 рублей, 
    зарегистрируйся на лэндинге 
    и участвую в розыгрыше ценных призов.
 
Во вложении файл «Розыгрыш Данные» с входными данными:
- Лист «Период розыгрыша» включает в себя информацию о всех клиентах, 
совершивших в период акции заказ от 5000 рублей 
(ид клиента, сегмент, РТО в период акции, кол-во заказов в период акции)

- Лист «До» включает в себя информацию о поведении клиентов из листа «Период розыгрыша» в период «до», 
аналогичный периоду розыгрыша (ид клиента, РТО в период акции, кол-во заказов в период акции)
- Лист «Участники» - это список клиентов, зарегистрировавшихся на лендинге и участвующих в розыгрыше.


SyntaxError: invalid syntax (Temp/ipykernel_2620/311882178.py, line 1)

In [3]:
#Все клиенты с заказом от 5000 рублей
segments = pd.read_excel('Розыгрыш_Данные.xlsx', sheet_name='период розыгрыша')
segments.columns = segments.loc[0]
segments = segments.iloc[1:,1:].set_index('client_id')
segments.head(5)

,segment,kol_z,rto
client_id,,,
12393,1 - core,1,9263
32568,1 - core,2,31881
46806,1 - core,1,5058
48703,3 - ottok,2,12904
56729,1 - core,10,36618


In [4]:
segments.reset_index().groupby('segment').agg({
    'client_id':'nunique',
    'kol_z':[np.sum, np.mean],
    'rto':[np.sum, np.mean]}
)

0         client_id  kol_z                  rto              
            nunique    sum      mean        sum          mean
segment                                                      
0 - new        1988   2856  1.436620   20666352  10395.549296
1 - core      30034  58043  1.932576  373019036  12419.891989
3 - ottok      9167  11379  1.241300   84134622   9177.988655

In [5]:
#Все клиенты с заказом от 5000 рублей до акции
clients_do = pd.read_excel('Розыгрыш_Данные.xlsx', sheet_name='до')
clients_do.columns = clients_do.loc[0]
clients_do = clients_do.iloc[1:,1:].set_index('client_id')
clients_do

,kol_z,rto
client_id,,
12393,1,13559
32568,2,14332
46806,1,4896
48703,1,8211
56729,11,36906
...,...,...
4204031,1,16063
4204045,1,26334
4204167,1,5432


In [6]:
#Клиенты, участвующие в розыгрыше
clients_prom = pd.read_excel('Розыгрыш_Данные.xlsx', sheet_name='участники')
clients_prom = clients_prom.iloc[:,1:].set_index('client_id')
clients_prom

,участие
client_id,
155433,1
157258,1
159113,1
184480,1
178885,1
...,...
4255361,1
4256293,1
4257246,1


In [7]:
total_clients = segments.join(clients_do, 
    how='left', 
    lsuffix='_prom', 
    rsuffix='_bef' 
).join(clients_prom,
       how='left'
      ).fillna(0)

total_clients['segment'] = total_clients['segment'].str.replace(' - ','_')

total_clients

,segment,kol_z_prom,rto_prom,kol_z_bef,rto_bef,участие
client_id,,,,,,
12393,1_core,1,9263,1,13559,0.0
32568,1_core,2,31881,2,14332,1.0
46806,1_core,1,5058,1,4896,0.0
48703,3_ottok,2,12904,1,8211,0.0
56729,1_core,10,36618,11,36906,0.0
...,...,...,...,...,...,...
4258893,0_new,1,9839,0,0,0.0
4259081,0_new,1,9402,0,0,0.0
4259095,0_new,1,9067,0,0,0.0


In [8]:
total_clients = total_clients[
    ['segment','участие', 
     'kol_z_bef','kol_z_prom', 
     'rto_bef','rto_prom']
].sort_values(by=['segment','участие','rto_bef','rto_prom'],
                                                 ascending=[True,False,True,True])

total_clients['AOV_bef'] = total_clients['rto_bef'].divide(total_clients['kol_z_bef']).fillna(0)
total_clients['AOV_prom'] = total_clients['rto_prom'].divide(total_clients['kol_z_prom']).fillna(0)

total_clients['kol_z_delt'] = total_clients['kol_z_prom'] - total_clients['kol_z_bef']
total_clients['AOV_delt'] = total_clients['AOV_prom'] - total_clients['AOV_bef']

total_clients.to_excel('Розыгрыш_Данные_Обработанные.xlsx', sheet_name='client_data')

In [18]:
total_clients.reset_index()

,client_id,segment,участие,kol_z_bef,kol_z_prom,rto_bef,rto_prom,AOV_bef,AOV_prom,kol_z_delt,AOV_delt,AOV_bef_r
0,4237905,0_new,1.0,0,1,0,5002,0.0,5002.0,1,5002.0,0.0
1,4226028,0_new,1.0,0,1,0,5012,0.0,5012.0,1,5012.0,0.0
2,4246598,0_new,1.0,0,1,0,5070,0.0,5070.0,1,5070.0,0.0
3,4256488,0_new,1.0,0,1,0,5073,0.0,5073.0,1,5073.0,0.0
4,4239564,0_new,1.0,0,1,0,5084,0.0,5084.0,1,5084.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
41184,2457744,3_ottok,0.0,1,1,46670,6149,46670.0,6149.0,0,-40521.0,46670.0
41185,4192242,3_ottok,0.0,2,1,46854,15199,23427.0,15199.0,-1,-8228.0,23427.0
41186,4193667,3_ottok,0.0,2,1,50790,13665,25395.0,13665.0,-1,-11730.0,25395.0
41187,1238235,3_ottok,0.0,1,1,56252,6970,56252.0,6970.0,0,-49282.0,56252.0


In [10]:
# Список клиентов для подбора похожих
total_clients['AOV_bef_r'] =  round( total_clients['AOV_bef'], 2)

total_clients_to_predict = total_clients.query('участие == 1').groupby(
    ['segment','kol_z_bef','AOV_bef_r','участие']
).count().reset_index()

total_clients_to_predict = total_clients_to_predict[ total_clients_to_predict.columns[:4] ]

total_clients_to_predict

,segment,kol_z_bef,AOV_bef_r,участие
0,0_new,0,0.00,1.0
1,1_core,0,0.00,1.0
2,1_core,1,1014.00,1.0
3,1_core,1,1077.00,1.0
4,1_core,1,1558.00,1.0
...,...,...,...,...
2205,3_ottok,2,7408.50,1.0
2206,3_ottok,2,9073.50,1.0
2207,3_ottok,2,13870.00,1.0
2208,3_ottok,3,4452.00,1.0


In [12]:
tot_frame = []
for client_n in total_clients_to_predict.index:
    client_dict = dict( total_clients_to_predict.reset_index().loc[client_n] )
    print(client_dict)
    
    total_clients_set = total_clients[ 
        total_clients['segment'] == client_dict['segment'] 
    ].query(f'(участие == 0)')
    
    #Вычислить дистанцию по среднему чеку и количеству заказов в период до промо
    total_clients_set['kol_z_bef__i'] = ( (
        total_clients_set['kol_z_bef'] - client_dict['kol_z_bef']
    ) / (
        total_clients_set['kol_z_bef'].max() - total_clients_set['kol_z_bef'].min()
    ) ).fillna(0)
    
    total_clients_set['AOV_bef__i'] = ( (
        total_clients_set['AOV_bef_r'] - client_dict['AOV_bef_r']
    ) / (
        total_clients_set['AOV_bef_r'].max() - total_clients_set['AOV_bef_r'].min()
    ) ).fillna(0)

    total_clients_set['Client_distance'] = 1 - np.sqrt( 
        total_clients_set['kol_z_bef__i']**2
        + total_clients_set['AOV_bef__i']**2 
    )
    
    #Дистанция - 1я квартиль
    distance_line = np.percentile(total_clients_set['Client_distance'], 80)
    line_set = total_clients_set[ total_clients_set['Client_distance'] >= distance_line ]
    
    mean_AOV = np.average(line_set['AOV_prom'], weights=line_set['Client_distance'])
    mean_kol_z = np.average(line_set['kol_z_prom'], weights=line_set['Client_distance'])
    
    client_dict['AOV_mean_0'] = mean_AOV
    client_dict['kol_z_mean_0'] = mean_kol_z
    
    tot_frame.append([
    client_dict['index'], 
    client_dict['segment'], 
    client_dict['участие'],
    client_dict['kol_z_bef'], 
    client_dict['AOV_bef_r'], 
    client_dict['AOV_mean_0'],
    client_dict['kol_z_mean_0']
    ])

predictions_set = pd.DataFrame( tot_frame, columns = ['indy','segment','участие','kol_z_bef','AOV_bef_r','AOV_mean_0','kol_z_mean_0'] )
predictions_set

{'index': 0, 'segment': '0_new', 'kol_z_bef': 0, 'AOV_bef_r': 0.0, 'участие': 1.0}
{'index': 1, 'segment': '1_core', 'kol_z_bef': 0, 'AOV_bef_r': 0.0, 'участие': 1.0}
{'index': 2, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 1014.0, 'участие': 1.0}
{'index': 3, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 1077.0, 'участие': 1.0}
{'index': 4, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 1558.0, 'участие': 1.0}
{'index': 5, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 1567.0, 'участие': 1.0}
{'index': 6, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 1616.0, 'участие': 1.0}
{'index': 7, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 1635.0, 'участие': 1.0}
{'index': 8, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 1679.0, 'участие': 1.0}
{'index': 9, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 1750.0, 'участие': 1.0}
{'index': 10, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 1845.0, 'участие': 1.0}
{'index': 11, 'segment': '1_core', 'kol_z_bef': 1

{'index': 101, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 3512.0, 'участие': 1.0}
{'index': 102, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 3531.0, 'участие': 1.0}
{'index': 103, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 3538.0, 'участие': 1.0}
{'index': 104, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 3540.0, 'участие': 1.0}
{'index': 105, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 3544.0, 'участие': 1.0}
{'index': 106, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 3549.0, 'участие': 1.0}
{'index': 107, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 3559.0, 'участие': 1.0}
{'index': 108, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 3569.0, 'участие': 1.0}
{'index': 109, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 3581.0, 'участие': 1.0}
{'index': 110, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 3606.0, 'участие': 1.0}
{'index': 111, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 3626.0, 'участие': 1.0}
{'index': 112, 'segme

{'index': 204, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 4385.0, 'участие': 1.0}
{'index': 205, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 4386.0, 'участие': 1.0}
{'index': 206, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 4392.0, 'участие': 1.0}
{'index': 207, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 4396.0, 'участие': 1.0}
{'index': 208, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 4402.0, 'участие': 1.0}
{'index': 209, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 4408.0, 'участие': 1.0}
{'index': 210, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 4410.0, 'участие': 1.0}
{'index': 211, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 4419.0, 'участие': 1.0}
{'index': 212, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 4421.0, 'участие': 1.0}
{'index': 213, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 4423.0, 'участие': 1.0}
{'index': 214, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 4427.0, 'участие': 1.0}
{'index': 215, 'segme

{'index': 297, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5054.0, 'участие': 1.0}
{'index': 298, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5057.0, 'участие': 1.0}
{'index': 299, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5070.0, 'участие': 1.0}
{'index': 300, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5073.0, 'участие': 1.0}
{'index': 301, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5079.0, 'участие': 1.0}
{'index': 302, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5087.0, 'участие': 1.0}
{'index': 303, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5101.0, 'участие': 1.0}
{'index': 304, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5112.0, 'участие': 1.0}
{'index': 305, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5120.0, 'участие': 1.0}
{'index': 306, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5123.0, 'участие': 1.0}
{'index': 307, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5135.0, 'участие': 1.0}
{'index': 308, 'segme

{'index': 393, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5851.0, 'участие': 1.0}
{'index': 394, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5862.0, 'участие': 1.0}
{'index': 395, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5865.0, 'участие': 1.0}
{'index': 396, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5867.0, 'участие': 1.0}
{'index': 397, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5879.0, 'участие': 1.0}
{'index': 398, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5902.0, 'участие': 1.0}
{'index': 399, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5906.0, 'участие': 1.0}
{'index': 400, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5925.0, 'участие': 1.0}
{'index': 401, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5927.0, 'участие': 1.0}
{'index': 402, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5944.0, 'участие': 1.0}
{'index': 403, 'segment': '1_core', 'kol_z_bef': 1, 'AOV_bef_r': 5956.0, 'участие': 1.0}
{'index': 404, 'segme

...

{'index': 1864, 'segment': '1_core', 'kol_z_bef': 4, 'AOV_bef_r': 4389.75, 'участие': 1.0}
{'index': 1865, 'segment': '1_core', 'kol_z_bef': 4, 'AOV_bef_r': 4411.0, 'участие': 1.0}
{'index': 1866, 'segment': '1_core', 'kol_z_bef': 4, 'AOV_bef_r': 4432.25, 'участие': 1.0}
{'index': 1867, 'segment': '1_core', 'kol_z_bef': 4, 'AOV_bef_r': 4524.75, 'участие': 1.0}
{'index': 1868, 'segment': '1_core', 'kol_z_bef': 4, 'AOV_bef_r': 4563.0, 'участие': 1.0}
{'index': 1869, 'segment': '1_core', 'kol_z_bef': 4, 'AOV_bef_r': 4670.0, 'участие': 1.0}
{'index': 1870, 'segment': '1_core', 'kol_z_bef': 4, 'AOV_bef_r': 4758.0, 'участие': 1.0}
{'index': 1871, 'segment': '1_core', 'kol_z_bef': 4, 'AOV_bef_r': 4827.75, 'участие': 1.0}
{'index': 1872, 'segment': '1_core', 'kol_z_bef': 4, 'AOV_bef_r': 4951.0, 'участие': 1.0}
{'index': 1873, 'segment': '1_core', 'kol_z_bef': 4, 'AOV_bef_r': 4956.5, 'участие': 1.0}
{'index': 1874, 'segment': '1_core', 'kol_z_bef': 4, 'AOV_bef_r': 4967.0, 'участие': 1.0}
{'inde

{'index': 1966, 'segment': '1_core', 'kol_z_bef': 11, 'AOV_bef_r': 3359.36, 'участие': 1.0}
{'index': 1967, 'segment': '1_core', 'kol_z_bef': 18, 'AOV_bef_r': 6700.11, 'участие': 1.0}
{'index': 1968, 'segment': '1_core', 'kol_z_bef': 22, 'AOV_bef_r': 2664.09, 'участие': 1.0}
{'index': 1969, 'segment': '1_core', 'kol_z_bef': 25, 'AOV_bef_r': 21022.88, 'участие': 1.0}
{'index': 1970, 'segment': '1_core', 'kol_z_bef': 42, 'AOV_bef_r': 17832.38, 'участие': 1.0}
{'index': 1971, 'segment': '1_core', 'kol_z_bef': 45, 'AOV_bef_r': 4627.76, 'участие': 1.0}
{'index': 1972, 'segment': '3_ottok', 'kol_z_bef': 0, 'AOV_bef_r': 0.0, 'участие': 1.0}
{'index': 1973, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 1314.0, 'участие': 1.0}
{'index': 1974, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 1341.0, 'участие': 1.0}
{'index': 1975, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 1518.0, 'участие': 1.0}
{'index': 1976, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 1542.0, 'участие':

{'index': 2057, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 5057.0, 'участие': 1.0}
{'index': 2058, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 5065.0, 'участие': 1.0}
{'index': 2059, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 5116.0, 'участие': 1.0}
{'index': 2060, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 5122.0, 'участие': 1.0}
{'index': 2061, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 5153.0, 'участие': 1.0}
{'index': 2062, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 5171.0, 'участие': 1.0}
{'index': 2063, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 5271.0, 'участие': 1.0}
{'index': 2064, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 5272.0, 'участие': 1.0}
{'index': 2065, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 5273.0, 'участие': 1.0}
{'index': 2066, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 5306.0, 'участие': 1.0}
{'index': 2067, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 5328.0, 'участие': 1.0}

{'index': 2159, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 11633.0, 'участие': 1.0}
{'index': 2160, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 11712.0, 'участие': 1.0}
{'index': 2161, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 11727.0, 'участие': 1.0}
{'index': 2162, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 11738.0, 'участие': 1.0}
{'index': 2163, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 12026.0, 'участие': 1.0}
{'index': 2164, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 12372.0, 'участие': 1.0}
{'index': 2165, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 12577.0, 'участие': 1.0}
{'index': 2166, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 13001.0, 'участие': 1.0}
{'index': 2167, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 13362.0, 'участие': 1.0}
{'index': 2168, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 13443.0, 'участие': 1.0}
{'index': 2169, 'segment': '3_ottok', 'kol_z_bef': 1, 'AOV_bef_r': 13556.0, 'уча

,indy,segment,участие,kol_z_bef,AOV_bef_r,AOV_mean_0,kol_z_mean_0
0,0,0_new,1.0,0,0.00,7821.562572,1.411406
1,1,1_core,1.0,0,0.00,7305.637138,1.420153
2,2,1_core,1.0,1,1014.00,7294.195304,1.427629
3,3,1_core,1.0,1,1077.00,7279.834333,1.430438
4,4,1_core,1.0,1,1558.00,7054.212913,1.513134
...,...,...,...,...,...,...,...
2205,2205,3_ottok,1.0,2,7408.50,7366.170491,1.229305
2206,2206,3_ottok,1.0,2,9073.50,7766.093735,1.221253
2207,2207,3_ottok,1.0,2,13870.00,8398.554104,1.218498
2208,2208,3_ottok,1.0,3,4452.00,6799.308033,1.272007


In [19]:
total_merged_clients = total_clients.reset_index().merge(predictions_set,
                    how='left',
                    left_on=['segment','kol_z_bef','AOV_bef_r','участие'], 
                    right_on=['segment','kol_z_bef','AOV_bef_r','участие'],
                    suffixes=('_left', '_right')
                                          )
total_merged_clients

,client_id,segment,участие,kol_z_bef,kol_z_prom,rto_bef,rto_prom,AOV_bef,AOV_prom,kol_z_delt,AOV_delt,AOV_bef_r,indy,AOV_mean_0,kol_z_mean_0
0,4237905,0_new,1.0,0,1,0,5002,0.0,5002.0,1,5002.0,0.0,0.0,7821.562572,1.411406
1,4226028,0_new,1.0,0,1,0,5012,0.0,5012.0,1,5012.0,0.0,0.0,7821.562572,1.411406
2,4246598,0_new,1.0,0,1,0,5070,0.0,5070.0,1,5070.0,0.0,0.0,7821.562572,1.411406
3,4256488,0_new,1.0,0,1,0,5073,0.0,5073.0,1,5073.0,0.0,0.0,7821.562572,1.411406
4,4239564,0_new,1.0,0,1,0,5084,0.0,5084.0,1,5084.0,0.0,0.0,7821.562572,1.411406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41184,2457744,3_ottok,0.0,1,1,46670,6149,46670.0,6149.0,0,-40521.0,46670.0,NaN,NaN,NaN
41185,4192242,3_ottok,0.0,2,1,46854,15199,23427.0,15199.0,-1,-8228.0,23427.0,NaN,NaN,NaN
41186,4193667,3_ottok,0.0,2,1,50790,13665,25395.0,13665.0,-1,-11730.0,25395.0,NaN,NaN,NaN
41187,1238235,3_ottok,0.0,1,1,56252,6970,56252.0,6970.0,0,-49282.0,56252.0,NaN,NaN,NaN


In [20]:
total_merged_clients.to_excel('Розыгрыш_Данные_Обработанные_Предсказанные.xlsx', sheet_name='predicted_data')